In [1]:
import pandas as pd
import pylab
import json
%matplotlib inline
import matplotlib.pyplot as plt
#plt.style.use('seaborn-whitegrid') # TODO: what did this do?
import numpy as np
from IPython.core import display
import psycopg2
import os
homedir = os.getenv("HOME")
with open(homedir + "/.pgpass",'r') as pgfile:
    pgdat = pgfile.read().strip().split(":")
    pgurl = pgdat[0]
    pgport = pgdat[1]
    pgdb = pgdat[2]
    pguser = pgdat[3]
    pgpass = pgdat[4]

%load_ext sql
from sqlalchemy import create_engine
# sql info: https://github.com/catherinedevlin/ipython-sql
# on error `pip install ipython-sql`
#uncomment for remote
#pgport=3333
pgport=5433
engine = create_engine('postgresql://postgres@localhost:%i/postgres' % pgport)
db = psycopg2.connect(host=pgurl, database=pgdb, user=pguser,password=pgpass,port=pgport)

In [2]:
def qry(q):
  return pd.read_sql_query(con = db, sql = q)
# pd.set_option("max_colwidth", None)
# pd.set_option('display.max_columns', None)
pd.set_option("display.latex.repr", False)
pd.set_option("display.latex.escape", True)

def iDisplay(v):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None, "max_colwidth", None):
        display.display(v)


Count number of uploaded APK files

In [3]:
%%sql postgresql://postgres@localhost:$pgport/postgres
select count(*) from apks where apkname not like 'jar_%' and apkname not like 'spec_%';

1 rows affected.


count
3


In [46]:
%%sql
select
    (select count(*) from apks) as apk_count, 
    (select count(*) from inputs) as input_count, 
    (select count(*) from jobs) as job_count, 
    (select count(*) from resultdata) as result_data_count, 
    (select count(*) from results) as result_count

 * postgresql://postgres@localhost:5433/postgres
1 rows affected.


apk_count,input_count,job_count,result_data_count,result_count
5,1,2226,1,1


Print jobs and statuses

In [91]:
%%sql
select 
    (select count(*) from jobs where status = 'completed') as Completed_Jobs,
    (select count(*) from jobs where status = 'paused') as Paused_Jobs,
    (select count(*) from jobs where status = 'acquired') as Acquired_jobs,
    (select count(id) from jobs 
       where stderr like '%Exception%no active body present for method%') as Body_Exception,
    (select count(id) from jobs where stderr like '%an implementation is missing%') as unimplemented,
    (select count(id) from jobs where stderr like '%Exception%') as Exception,
    (select count(id) from jobs where stderr like '%no libz3java in java.library.path%') as path_exception,
    (select count(id) from jobs where stderr like '%Failed to parse reflective type reference%') as Refl_exception,
    (select count(id) from jobs where status like 'failed%') as worker_failure,
    (select count(id) from jobs where stdout like '%empty pred location%') as possible_cg_unsound,
    (select count(*) from jobs) as Total_Jobs

 * postgresql://postgres@localhost:5433/postgres
1 rows affected.


completed_jobs,paused_jobs,acquired_jobs,body_exception,unimplemented,exception,path_exception,refl_exception,worker_failure,possible_cg_unsound,total_jobs
151,0,6,0,4,0,0,0,0,0,2226


In [74]:
# find exceptions
#%%sql 
#select replace(cast(stderr as varchar),'$','\$') from jobs where stderr like '%Exception%' limit 1;
out = %sql select stderr,config from jobs where stderr like '%Exception%' order by random() limit 1;
if(len(out) != 0):
    print(out[0][1])
    print(out[0][0])
else:
    print("none")


 * postgresql://postgres@localhost:5433/postgres
0 rows affected.
none


In [75]:
# find exceptions
#%%sql 
#select replace(cast(stderr as varchar),'$','\$') from jobs where stderr like '%Exception%' limit 1;
out = %sql select stderr,config from jobs where stderr like '%an implementation is missing%' order by random() limit 1;
if(len(out) != 0):
    print(out[0][1])
    print(out[0][0])
else:
    print("none")


 * postgresql://postgres@localhost:5433/postgres
1 rows affected.
{"apkPath":"${baseDir}/sgtpuzzles/app/build/outputs/apk/debug/app-debug.apk","outFolder":["${baseDirOut}/name.boyle.chris/name.boyle.chris.sgtpuzzles.PrefsActivity$PrefsMainFragment"],"initialQuery":[{"t":"AllReceiversNonNull","className":"name.boyle.chris.sgtpuzzles.PrefsActivity$PrefsMainFragment"}],"samples":90,"timeLimit":180}
scala.NotImplementedError: an implementation is missing
	at scala.Predef$.$qmark$qmark$qmark(Predef.scala:344)
	at edu.colorado.plv.bounder.symbolicexecutor.TransferFunctions.cmdTransfer(TransferFunctions.scala:838)
	at edu.colorado.plv.bounder.symbolicexecutor.TransferFunctions.transfer(TransferFunctions.scala:219)
	at edu.colorado.plv.bounder.symbolicexecutor.AbstractInterpreter.$anonfun$executeStep$1(AbstractInterpreter.scala:552)
	at scala.collection.immutable.List.flatMap(List.scala:293)
	at scala.collection.immutable.List.flatMap(List.scala:79)
	at edu.colorado.plv.bounder.symbolicexecuto

Get failing apks

In [58]:
%%sql
select cast(config::json->'apkPath' as text) from jobs 
        where stderr like '%Exception%' group by cast(config::json->'apkPath' as text) ;

 * postgresql://postgres@localhost:5433/postgres
0 rows affected.


text


In [59]:
%%sql
select cast(config::json->'apkPath' as text) as app,count(*) as count from jobs group by app order by count desc;

 * postgresql://postgres@localhost:5433/postgres
3 rows affected.


app,count
"""${baseDir}/AntennaPod/app/build/outputs/apk/free/debug/app-free-debug.apk""",1864
"""${baseDir}/connectbot/app/build/outputs/apk/google/debug/app-google-debug.apk""",235
"""${baseDir}/sgtpuzzles/app/build/outputs/apk/debug/app-debug.apk""",127


In [68]:
%%sql
select count(*),status from jobs group by status;

 * postgresql://postgres@localhost:5433/postgres
3 rows affected.


count,status
48,completed
2172,new
6,acquired


In [82]:
%%sql 
select cast(result::json->'summary' as varchar) as summary, cast(result::json->'maxPathCh' as varchar) as ch,count(*),avg(querytime) from results group by summary,ch order by summary,ch;

 * postgresql://postgres@localhost:5433/postgres
4 rows affected.


summary,ch,count,avg
"""\""Ian implementation is missing\""""","""\""UnknownCharacterization\""""",6,1.5000000000000000
"""\""Itimeout\""""","""\""UnknownCharacterization\""""",161,181.0000000000000000
"""\""Unreachable\""""","""\""UnknownCharacterization\""""",1269,0.06540583136327817179
"""\""Witnessed\""""","""\""UnknownCharacterization\""""",63,0.77777777777777777778


In [62]:
%%sql
select id,cast(result::json->'summary' as varchar) as summary,loc::json->'method'->>1 as clazz,loc::json->'method'->>2 from results order by random() limit 5;

 * postgresql://postgres@localhost:5433/postgres
5 rows affected.


id,summary,clazz,?column?
15152,"""\""Unreachable\""""",name.boyle.chris.sgtpuzzles.NightModeHelper,"[""name.boyle.chris.sgtpuzzles.NightModeHelper""]"
15144,"""\""Unreachable\""""",name.boyle.chris.sgtpuzzles.NightModeHelper,"[""name.boyle.chris.sgtpuzzles.NightModeHelper"",""android.hardware.SensorEvent""]"
15009,"""\""Unreachable\""""",name.boyle.chris.sgtpuzzles.databinding.KeyboardBorderedBinding,"[""name.boyle.chris.sgtpuzzles.databinding.KeyboardBorderedBinding""]"
15091,"""\""Unreachable\""""",name.boyle.chris.sgtpuzzles.NightModeHelper,"[""name.boyle.chris.sgtpuzzles.NightModeHelper""]"
15081,"""\""Unreachable\""""",name.boyle.chris.sgtpuzzles.GamePlay$$ExternalSyntheticLambda0,"[""name.boyle.chris.sgtpuzzles.GamePlay$$ExternalSyntheticLambda0"",""android.content.DialogInterface""]"


Triage witness results
----------------------

In [63]:
witnessesq = %sql \
select id,jobid,resultdata,loc::json->'method'->>1 as clazz,loc::json->'method'->>2 from results \
                where cast(result::json->'summary' as varchar) like '%Witnessed%' order by id asc limit 100;

witnessesdf = witnessesq.DataFrame()
witnessesdf.to_csv('triageWit.csv')

 * postgresql://postgres@localhost:5433/postgres
7 rows affected.


In [64]:
witnessesq = %sql \
select id,jobid,resultdata,loc::json->'method'->>1 as clazz,loc::json->'method'->>2 from results \
                where cast(result::json->'summary' as varchar) like '%Itimeout%' and cast(result::json->'maxPathCh' as varchar) like '%MultiCallback%' order by id asc limit 100;

witnessesdf = witnessesq.DataFrame()
witnessesdf.to_csv('triageWitMultiTimeout.csv')

 * postgresql://postgres@localhost:5433/postgres
0 rows affected.


In [ ]:
%%sql 
select id,jobid,resultdata,loc::json->'method'->>1 as clazz, 
        replace(cast(loc::json->'line'->'str' as varchar),'$','\$') from results where id = 40;

Triage proven results
--------------------------

In [ ]:
%%sql 
select id,jobid,resultdata,loc::json->'method'->>1 as clazz,loc::json->'method'->>2 from results 
                where cast(result::json->'summary' as varchar) like '%Proven%' order by random() limit 10;

Triage unreachable results
--------------------------

In [ ]:
%%sql 
select id,jobid,resultdata,loc::json->'method'->>1 as clazz 
            from results where cast(result::json->'summary' as varchar) like '%Unreachable%' 
            order by random() limit 15;

In [ ]:
%%sql
select id,jobid,resultdata,loc::json->'method'->>1 as clazz, 
        replace(cast(loc::json->'method'->>0 as varchar),'$','\$') as mname, 
        replace(cast(loc::json->'line' as varchar),'$','\$') as location 
        from results where id = 15037;

In [ ]:
%%sql
select id,result,replace(cast(loc::json->'line' as varchar),'$','\$') as location 
    from results where loc like '%244 $r3%';

In [ ]:
%%sql
select replace(cast(loc as varchar),'$','\$') from results where id = 1494;

Runtime Stats
=============

In [ ]:
queryTime = %sql select querytime from results;
tplt = queryTime.DataFrame().plot.hist(bins=32)
# tplt.set_yscale('log')
tplt.set_xlabel('time (sec)')
# split this into proven and witnessed

In [ ]:
# Job runtimes
runtimes = %sql select ended - started as time from jobs where ended is not NULL;
runtimesDF = runtimes.DataFrame()
runtimesDF['time'] = runtimesDF['time'].apply(lambda x: x.total_seconds() / (60))
tplt = runtimesDF.plot.hist(bins=32)
tplt.set_yscale('log')
tplt.set_xlabel('time (min)')

In [ ]:
# Plot job time trend
rtvt = %sql select started, ended - started as time from jobs where ended is not NULL;
rtvtDF = rtvt.DataFrame()
rtvtDF.sort_values('started', ascending=True)
fig, ax = plt.subplots()
ax.plot_date(rtvtDF['started'], rtvtDF['time'], marker='*', linestyle='')

fig.autofmt_xdate()
plt.show()


Total job time

In [ ]:
%%sql
select sum(ended-started) from jobs where ended is not NULL;

In [ ]:
%%sql
select sum(querytime)/60./60. from results;

In [ ]:
%%sql
select now() - min(started) from jobs;

Size of postgres data
=====================

In [ ]:
%%sql
WITH RECURSIVE pg_inherit(inhrelid, inhparent) AS
    (select inhrelid, inhparent
    FROM pg_inherits
    UNION
    SELECT child.inhrelid, parent.inhparent
    FROM pg_inherit child, pg_inherits parent
    WHERE child.inhparent = parent.inhrelid),
pg_inherit_short AS (SELECT * FROM pg_inherit WHERE inhparent NOT IN (SELECT inhrelid FROM pg_inherit))
SELECT table_schema
    , TABLE_NAME
    , row_estimate
    , pg_size_pretty(total_bytes) AS total
    , pg_size_pretty(index_bytes) AS INDEX
    , pg_size_pretty(toast_bytes) AS toast
    , pg_size_pretty(table_bytes) AS TABLE
  FROM (
    SELECT *, total_bytes-index_bytes-COALESCE(toast_bytes,0) AS table_bytes
    FROM (
         SELECT c.oid
              , nspname AS table_schema
              , relname AS TABLE_NAME
              , SUM(c.reltuples) OVER (partition BY parent) AS row_estimate
              , SUM(pg_total_relation_size(c.oid)) OVER (partition BY parent) AS total_bytes
              , SUM(pg_indexes_size(c.oid)) OVER (partition BY parent) AS index_bytes
              , SUM(pg_total_relation_size(reltoastrelid)) OVER (partition BY parent) AS toast_bytes
              , parent
          FROM (
                SELECT pg_class.oid
                    , reltuples
                    , relname
                    , relnamespace
                    , pg_class.reltoastrelid
                    , COALESCE(inhparent, pg_class.oid) parent
                FROM pg_class
                    LEFT JOIN pg_inherit_short ON inhrelid = oid
                WHERE relkind IN ('r', 'p')
             ) c
             LEFT JOIN pg_namespace n ON n.oid = c.relnamespace
  ) a
  WHERE oid = parent
) a
ORDER BY total_bytes DESC limit 5;

Inspection of exceptions
========================

In [ ]:
%%sql
select id,config::json->'apkPath',replace(stderr,'$','\$') from jobs where stderr like '%Exception%';

In [ ]:
#count of total results
%sql select count(*) as Total_locations_computed from results